In [1]:
import pandas as pd

df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

not_fit = True


In [2]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
nltk.download('stopwords')
stop = stopwords.words('english')

def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]

[nltk_data] Downloading package stopwords to /Users/AtSu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import numpy as np
from bs4 import BeautifulSoup
from collections import defaultdict
import src.proc_data as procd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
"""
train_channel = np.zeros((df.values.shape[0], 4), dtype=object)
test_channel = np.zeros((df_test.values.shape[0], 4), dtype=object)
columns = ['entertainment', 'world', 'social-media', 'tech']


for i in range(df['Page content'].values.shape[0]):
    channel = procd.parse_channel(df['Page content'][i])
    if channel in columns:
        train_channel[i, columns.index(channel)] = 1

pd.DataFrame(data=train_channel, columns=columns).to_csv('train_channel.csv', index=False)
        
for i in range(df_test['Page content'].values.shape[0]):
    channel = procd.parse_channel(df_test['Page content'][i])
    if channel in columns:
        test_channel[i, columns.index(channel)] = 1
        
pd.DataFrame(data=test_channel, columns=columns).to_csv('test_channel.csv', index=False)
"""
"""
train_parse = np.zeros((df.values.shape[0], 5),dtype=object)

for i in range(df['Page content'].values.shape[0]):
#for i in range(5):

    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3]\
            = procd.parse_html(df['Page content'][i])
    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))

pd.DataFrame(data=train_parse, columns=['title', 'content', 'author', 'n_fig', 'n_words']).to_csv('train_b_parsed.csv')

train_parse = np.zeros((df_test.values.shape[0], 5),dtype=object)

#for i in range(5):

for i in range(df_test['Page content'].values.shape[0]):
    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3] \
            = procd.parse_html(df_test['Page content'][i])
    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))

pd.DataFrame(data=train_parse, columns=['title', 'content', 'author','n_fig', 'n_words']).to_csv('test_b_parsed.csv')
"""

"\ntrain_parse = np.zeros((df.values.shape[0], 5),dtype=object)\n\nfor i in range(df['Page content'].values.shape[0]):\n#for i in range(5):\n\n    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3]            = procd.parse_html(df['Page content'][i])\n    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))\n\npd.DataFrame(data=train_parse, columns=['title', 'content', 'author', 'n_fig', 'n_words']).to_csv('train_b_parsed.csv')\n\ntrain_parse = np.zeros((df_test.values.shape[0], 5),dtype=object)\n\n#for i in range(5):\n\nfor i in range(df_test['Page content'].values.shape[0]):\n    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3]             = procd.parse_html(df_test['Page content'][i])\n    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))\n\npd.DataFrame(data=train_parse, columns=['title', 'content', 'author','n_fig', 'n_words']).to_csv('test_b_parsed.csv')\n"

In [4]:
df = pd.read_csv('train_parsed.csv')
df_test = pd.read_csv('test_parsed.csv')


In [5]:
from scipy.sparse import hstack
from sklearn.preprocessing import PolynomialFeatures

train_weekday = pd.read_csv('train_weekday.csv').values
train_time = pd.read_csv('train_time.csv').values
train_channel = pd.read_csv('train_channel.csv').values

poly = PolynomialFeatures(degree=3)
X_train_aug = np.concatenate((train_channel, train_weekday, train_time, df['n_fig'].reshape(-1,1)), axis=1)
X_train_aug = poly.fit_transform(X_train_aug)[:, 1:]

print(X_train_aug.shape)

(27643, 559)


In [6]:
from sklearn.model_selection import train_test_split
# hold out testing set
y_train = df['Popularity']
X_htrain, X_htest, y_htrain, y_htest = train_test_split(X_train_aug, y_train, test_size=0.3, random_state=0)


In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

SGD = LogisticRegression(C=3.0, random_state=0)
SGD.fit(X_train_aug, y_train)

pred = SGD.predict(X_htrain)
pred_t = SGD.predict(X_htest)
score_train = metrics.accuracy_score(y_htrain, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)

# best case here
print("SGD score (train): %f, (test): %f"%(score_train, score_test))
candidates = []
candidates.append({'clf':SGD, 'C':3.0, 'score':0.557699})


SGD score (train): 0.550543, (test): 0.557699


In [8]:
test_weekday = pd.read_csv('test_weekday.csv').values
test_time = pd.read_csv('test_time.csv').values
test_channel = pd.read_csv('test_channel.csv').values


X_test_con = np.concatenate((test_channel, test_weekday, test_time, df_test['n_fig'].reshape(-1,1)), axis=1)
X_test_con = poly.transform(X_test_con)[:,1:]

In [9]:

for cand in candidates:
    predict = cand['clf'].predict(X_test_con)
    output = np.zeros((X_test_con.shape[0], 2), dtype=int)
    output[:, 0] = df_test['Id']
    output[:, 1] = predict
    df_output = pd.DataFrame(data=output, columns=['Id', 'Popularity'])
    df_output.to_csv('test___%f.csv' % (cand['C']), index=False)